- Find the oldest product
- Find the newest product
- Average product age, where age means how long the product has been in the system.
- List of other countries where products are sold too.
- Identify category of products and the compute:
   Number of products by category
    List containing names of products by category 
- Identify traces and compute:
    Number of products by trace
    List containing names of products by trace
- Data quality analysis on fields of interest (see appendix 1):
   Number of products with complete info.
    % of products without complete analysis per 100g
    % of products without additives info
    % of products without traces info
- Data profiling on fields of interest (see appendix 1):
    Stats on analysis per 100g fields

In [18]:
#Importing APIs
import pandas as pd
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from pyspark.sql.window import Window
from IPython.display import display, Markdown
from pyspark.sql import SQLContext


sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sqlContext = SQLContext(spark)

In [19]:
#Load dataset
resto = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", "\t") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "false") \
                 .csv("noTT.csv")
tt = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", ";") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("title.csv")

In [20]:
resto.show(3)
rest = resto.drop("C_185")
#.drop("C_173").drop("C_174").drop("C_175")
rest.show(1)
tt.show(1)
concate = tt.union(rest)

+----------+--------------------+-------------+----------+-------------------+----------+-------------------+--------------------+----+----------------+-----+---------+---------+----+------------------+------------------+--------------------+--------------------+--------------------+----+----+----+-------+-------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+-----------+----------+-------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+-----+----+----+----+--------------------+--------------------+----+----+----+----+----+----+----+----+----+-------------+-------+--------------------+--------------------+--------------------+----+--------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-----+------+----+----+---------------+----+----+----+----+----+----+----+----+--

In [21]:
concate.show(2)

+----+--------------------+-------------+----------+-------------------+---------------+----------------------+--------------------+------------------------+------------+--------+---------+--------------+--------------+------------------+------------------+-----------+---------------+-------------+-------+------------+----+--------------------+-------------------------+-------------+----------------+-------------+--------------+-------------------------+------+-----------+----+---------------+------+---------+--------------+------------+----------------+---------+--------------+------+-----------+---------+------------+----------------+------------+-----------+----+--------------+------------+---------------------------+-------------------------+------------------------------+---------------------------------------+-------------------------------------+------------------------------------------+----------------+----------------+----------+-------------+-------------+-------------------

In [22]:
pdcont = concate.toPandas()

In [23]:
pd.set_option('display.max_columns', None)
pdcont.head(30)

code                                                url  \
0            17.0  http://world-en.openfoodfacts.org/product/0000...   
1             5.0  http://world-en.openfoodfacts.org/product/0000...   
2      2.042464E7  http://world-en.openfoodfacts.org/product/0000...   
3     2.9035175E7  http://world-en.openfoodfacts.org/product/0000...   
4     2.9066926E7  http://world-en.openfoodfacts.org/product/0000...   
5     4.2116424E7  http://world-en.openfoodfacts.org/product/0000...   
6     4.2116455E7  http://world-en.openfoodfacts.org/product/0000...   
7     6.9008078E7  http://world-en.openfoodfacts.org/product/0000...   
8     6.9009037E7  http://world-en.openfoodfacts.org/product/0000...   
9     6.9014901E7  http://world-en.openfoodfacts.org/product/0000...   
10    6.9015335E7  http://world-en.openfoodfacts.org/product/0000...   
11    6.9114144E7  http://world-en.openfoodfacts.org/product/0000...   
12    6.9114724E7  http://world-en.openfoodfacts.org/product/0000...   
13      8.01768E7  http://world-en.openfoodfacts.org/product/0000...   
14          949.0  http://world-en.openfoodfacts.org/product/0000...   
15         1007.0  http://world-en.openfoodfacts.org/product/0000...   
16   2.04286484E8  http://world-en.openfoodfacts.org/product/0000...   
17   2.50632969E8  http://world-en.openfoodfacts.org/product/0000...   
18   4.60938714E8  http://world-en.openfoodfacts.org/product/0000...   
19     4.703228E8  http://world-en.openfoodfacts.org/product/0000...   
20   5.01050603E8  http://world-en.openfoodfacts.org/product/0000...   
21   5.26938306E8  http://world-en.openfoodfacts.org/product/0000...   
22   7.90310013E8  http://world-en.openfoodfacts.org/product/0000...   
23    7.9031002E8  http://world-en.openfoodfacts.org/product/0000...   
24            1.0  http://world-en.openfoodfacts.org/product/0001...   
25           11.0  http://world-en.openfoodfacts.org/product/0001...   
26  1.100000136E9  http://world-en.openfoodfacts.org/product/0001...   
27   1.12640151E8  http://world-en.openfoodfacts.org/product/0001...   
28  1.182000086E9  http://world-en.openfoodfacts.org/product/0001...   
29        15257.0  http://world-en.openfoodfacts.org/product/0001...   

                       creator   created_t    created_datetime   \
0                      kiliweb  1591989744  2020-06-12 21:22:24   
1                waistline-app  1559391422  2019-06-01 14:17:02   
2                    twoflower  1489527029  2017-03-14 22:30:29   
3                waistline-app  1549218790  2019-02-03 19:33:10   
4                waistline-app  1549274985  2019-02-04 11:09:45   
5              date-limite-app  1533734197  2018-08-08 15:16:37   
6              date-limite-app  1533734066  2018-08-08 15:14:26   
7                      kiliweb  1523034665  2018-04-06 19:11:05   
8              date-limite-app  1544958504  2018-12-16 12:08:24   
9              date-limite-app  1549297076  2019-02-04 17:17:56   
10             date-limite-app  1535207159  2018-08-25 16:25:59   
11             date-limite-app  1549295935  2019-02-04 16:58:55   
12             date-limite-app  1544958793  2018-12-16 12:13:13   
13  openfoodfacts-contributors  1548684719  2019-01-28 15:11:59   
14  openfoodfacts-contributors  1603063058  2020-10-19 01:17:38   
15  openfoodfacts-contributors  1603063094  2020-10-19 01:18:14   
16        allfitnessfactory-de  1483099966  2016-12-30 13:12:46   
17        allfitnessfactory-de  1484292612  2017-01-13 08:30:12   
18        allfitnessfactory-de  1483097990  2016-12-30 12:39:50   
19        allfitnessfactory-de  1484302932  2017-01-13 11:22:12   
20        allfitnessfactory-de  1484302351  2017-01-13 11:12:31   
21        allfitnessfactory-de  1484301827  2017-01-13 11:03:47   
22                    malikele  1388682187  2014-01-02 18:03:07   
23                    malikele  1388661711  2014-01-02 12:21:51   
24             date-limite-app  1531760498  2018-07-16 19:01:38   
25             date-limite-app  158332

In [24]:
pdcont.describe()

code                                                url  \
count    76055                                              76055   
unique   76032                                              76055   
top     1007.0  http://world-en.openfoodfacts.org/product/4004...   
freq         3                                                  1   

                           creator   created_t    created_datetime   \
count                        76055       76055                76055   
unique                        2083       75673                75673   
top     openfoodfacts-contributors  1561571298  2019-06-26 19:48:18   
freq                         14192           6                    6   

       last_modified_t last_modified_datetime  product_name  \
count            76055                  76055         64107   
unique           73061                  73061         49471   
top         1607376555    2020-12-07 22:29:15  Kichererbsen   
freq                13                     13            59   

               abbreviated_product_name  \
count                                 1   
unique                                1   
top     Bq 2 blancs plt panes s/at gaul   
freq                                  1   

                                             generic_name quantity  \
count                                               12624    58134   
unique                                              10077     4342   
top     Natürliches Mineralwasser mit Kohlensäure vers...    200 g   
freq                                                   73     2500   

         packaging packaging_tags packaging_text brands brands_tags  \
count        40624          40623            385  61766       61759   
unique        8092           6882            237  16041       13607   
top     Kunststoff     kunststoff   Grüner Punkt  Edeka       edeka   
freq          5335           5578             12    956        1066   

       categories categories_tags categories_en      origins origins_tags  \
count       47082           47081         47081         4843         4835   
unique      15055            9544          9544         1397         1249   
top     Beverages    en:beverages     Beverages  Deutschland   en:germany   
freq         1087            1354          1354          906         1013   

             ?? manufacturing_places manufacturing_places_tags labels  \
count      4835                10504                     10502  25564   
unique     1247                 3600                      3392   9727   
top     Germany          Deutschland               deutschland    Bio   
freq       1013                 2211                      2224   1124   

       labels_tags emb_codes emb_codes_tags first_packaging_code_geo   \
count        25563     25563           8358                      8357   
unique        7666      7664           3962                      3027   
top     en:organic   Organic    FSC-C014047               fsc-c014047   
freq          1639      1639            139                       140   

                     cities cities_tags  \
count                   396           0   
unique                  169           0   
top     47.816667,-0.683333         NaN   
freq                     18         NaN   

                                               ??? purchase_places stores  \
count                                          437           15518  18698   
unique                                         215            1585   1764   
top     tilloy-les-mofflaines-pas-de-calais-france     Deutschland   Lidl   
freq                                            14            6466   1990   

         countries countries_tags countries_en  \
count        76055          76055        76055   
unique        2413           1029         1029   
top     en:Germany     en:germany      Germany   
freq         17572          46638        46638   

                                         ingredients_text allergens  \
count                      

In [25]:
#pdfood = pd.read_csv("noT.csv", sep = "\t")

In [26]:
#checking the right position of the columns
concate.filter(col("ingredients_text") != "").select("ingredients_text").show(25)

+--------------------+
|    ingredients_text|
+--------------------+
|Zucker, Kakaomass...|
|Zucker,Palmöl, Ha...|
|Proteinmischung (...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|sugar, corn syrup...|
|sugar, corn syrup...|
|Natürliches Miner...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Enthält: 10 mg Me...|
|Tapioca Syrup, Ve...|
|         quinoa 100%|
|Sojaproteinisolat...|
|Heringsfilets (88...|
|Cucumbers, water,...|
|Composition pour ...|
|corn syrup, sugar...|
|water, soybean oi...|
|Branntweinessig, ...|
|Distilled Vinegar...|
|Branntweinessig, ...|
+--------------------+
only showing top 25 rows



Creator
Created_datetime
Last_modified_datetime
Product_name
Countries_en
Traces_en
Additives_tags
Main_category_en
Image_url
Quantity
Packaging_tags
Categories_en
Ingredients_text
Additives_en
Energy-kcal_100g
Fat_100g
Saturated-fat_100g
Sugars_100g
Salt_100g/sodium_100g

In [57]:
prepared_df = concate.select("Creator","created_t",
"Last_modified_t",
"Product_name",
"countries_en",
"traces_en",
"additives_en",
"Main_category",
"Image_url",
"Quantity",
"Packaging_tags",
"Categories",
"Ingredients_text",
"Additives_n",
"energy-kj_100g",
"Fat_100g",
"Saturated-fat_100g",
"Sugars_100g",
"Salt_100g")
#note: could not find additive, replaced with additive_n

In [28]:
prepared_df.show(3)

+-------------+----------+---------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+-----------+--------------+--------+------------------+-----------+---------------+
|      Creator| created_t|Last_modified_t|        Product_name|countries_en|           traces_en|      Additives_tags|       Main_category|           Image_url|Quantity|Packaging_tags|          Categories|    Ingredients_text|Additives_n|energy-kj_100g|Fat_100g|Saturated-fat_100g|Sugars_100g|      Salt_100g|
+-------------+----------+---------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+-----------+--------------+--------+------------------+-----------+---------------+
|      kiliweb|1591989744|     1609478763|    Vitória crackers|     Ge

In [29]:
prepared_df.filter(col("Salt_100g") != "").select("Salt_100g").count()

45460

# Subset data analysis in Spark

## Find the oldest product

In [30]:
prepared_df.select("created_t", "product_name").sort(col("created_t"),ascending = False).show(1)

+----------+--------------+
| created_t|  product_name|
+----------+--------------+
|1612662755|Sandwich Sauce|
+----------+--------------+
only showing top 1 row



## Find the newest product

In [31]:
prepared_df.select("created_t", "Product_name").sort(col("created_t"),ascending = True).show(1)

+----------+--------------------+
| created_t|        Product_name|
+----------+--------------------+
|1329035567|Milka Noisette En...|
+----------+--------------------+
only showing top 1 row



## Average product age, where age means how long the product has been in the system.

In [32]:
#convertion from unix format seconds to days to years = 3600*24*365
prepared_df.filter(col("created_t") > 0 ).agg((avg("created_t")/31536000).alias("average time (years)")).show()

+--------------------+
|average time (years)|
+--------------------+
|   49.15678858316572|
+--------------------+



## List of other countries where products are sold too.

In [33]:
prepared_df.select(explode(split(col("countries_en"), ",")).alias("indivcountry"))  \
            .filter((col("indivcountry") != "Allemagne") & (col("indivcountry") != "Germany") & (col("indivcountry") != "Deutschland") & ( col("indivcountry") != "en:DE") & ( col("indivcountry") != "en:de")) \
            .select("indivcountry").distinct() \
            .show(15)

+--------------------+
|        indivcountry|
+--------------------+
|  Middle-east-africa|
|       Côte d'Ivoire|
|          Luxemburgo|
|             Andorre|
|                Inde|
|Czech-republic-če...|
|     Greece-ελληνικά|
|              Russia|
|          nl:pologne|
|      Estados-unidos|
|      Hungary-magyar|
|      Romania-romană|
|      Hong-kong-粵語|
|       Malaysia-中文|
|               Suiza|
+--------------------+
only showing top 15 rows



## Identify traces and compute: Number of products by trace List containing names of products by trace

In [34]:
prepared_df.filter(col("traces_en") != 'null').groupby("traces_en") \
            .agg(count("Product_name").alias("product_total")) \
            .sort("product_total",ascending =False) \
            .show(40)

+--------------------+-------------+
|           traces_en|product_total|
+--------------------+-------------+
|                Nuts|         1319|
|        Nuts,Peanuts|          464|
|            Soybeans|          457|
|                Milk|          441|
|      Celery,Mustard|          437|
|        Sesame seeds|          321|
|         Gluten,Nuts|          246|
|              Celery|          245|
|              Gluten|          233|
|           Milk,Nuts|          223|
|       Nuts,Soybeans|          203|
|                Eggs|          177|
|             Mustard|          152|
|   Nuts,Sesame seeds|          147|
|   Eggs,Nuts,Peanuts|          127|
|Eggs,Gluten,Nuts,...|          121|
|       Eggs,Soybeans|          119|
|Sulphur dioxide a...|          117|
|Nuts,Sesame seeds...|          104|
|  Milk,Nuts,Soybeans|           99|
|Milk,Nuts,Sesame ...|           97|
|Milk,Nuts,Peanuts...|           93|
|Celery,Mustard,So...|           90|
|       Milk,Soybeans|           83|
|

## Data quality analysis on fields of interest (see appendix 1): Number of products with complete info. % of products without complete analysis per 100g % of products without additives info % of products without traces info

In [40]:
prepared_df.filter((col("Creator").isNotNull()) & 
                   (col("Last_modified_t").isNotNull()) &
                  (col("Product_name").isNotNull()) & 
                   (col("countries_en").isNotNull()) & 
                   (col("traces_en").isNotNull()) &
                   (col("additives_en").isNotNull()) & 
                   (col("Packaging_tags").isNotNull()) & 
                   (col("Categories").isNotNull()) & 
                   (col("Ingredients_text").isNotNull()) & 
                   (col("Energy-kcal_100g").isNotNull()) & 
                   (col("Saturated-fat_100g").isNotNull()) & 
                   (col("Sugars_100g").isNotNull()) & 
                   (col("Salt_100g").isNotNull()) ).count()


0

In [41]:
productsid = prepared_df.select(col("created_t")).count()
null100 = prepared_df.filter((col("Energy-kcal_100g").isNotNull()) & 
                   (col("Saturated-fat_100g").isNotNull()) & 
                   (col("Sugars_100g").isNotNull()) & 
                   (col("Salt_100g").isNotNull()) ).count()
(100 * null100 / productsid)

0.0

In [42]:
nulladd = prepared_df.filter( col("additives_en").isNull()).count()
print("Ratio of null additive to all product")
(100 * nulladd / productsid)

Ratio of null additive to all product


78.4366576819407

In [43]:
nulltrace = prepared_df.filter( col("traces_en").isNull()).count()
print("Ratio of null traces to all product")
(100 * nulltrace / productsid)

Ratio of null traces to all product


83.98790349089475

In [39]:
#dictanalysis = [prepared_df.filter(col(column).isNotNull()).count() for column in prepared_df.schema.names]

## Data profiling on fields of interest (see appendix 1): Stats on analysis per 100g fields

## Identify which products are safe.

In [45]:
#Authorized additive in the eu 
#Retrieved from https://www.food.gov.uk/business-guidance/approved-additives-and-e-numbers
additives = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", ";") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("authorised_additive_eu.csv")
additives.show()

+---------+--------------------+
|E numbers|           Additives|
+---------+--------------------+
|     E100|            Curcumin|
|     E101|      (i) Riboflavin|
|     null|(ii) Riboflavin-5...|
|     E102|          Tartrazine|
|     E104|    Quinoline yellow|
|     E110|Sunset Yellow FCF...|
|     E120|Cochineal; Carmin...|
|     E122|Azorubine; Carmoi...|
|     E123|            Amaranth|
|     E124|Ponceau 4R; Cochi...|
|     E127|         Erythrosine|
|     E129|       Allura Red AC|
|     E131|       Patent Blue V|
|     E132|lndigotine; Indig...|
|     E133|  Brilliant Blue FCF|
|     E140|Chlorophylls and ...|
|     E141|Copper complexes ...|
|     E142|             Green S|
|    E150a|       Plain caramel|
|    E150b|Caustic sulphite ...|
+---------+--------------------+
only showing top 20 rows



In [66]:
aditive_df = prepared_df.select(explode(split(col("additives_en"), ",")).alias("additives_list"), "Product_name").withColumn("E numbers", regexp_extract("additives_list", r"E\d+",0))
health_df = aditive_df.join(additives,"E numbers",how="left")

health_df.show(3)

+---------+----------------+------------------+------------+
|E numbers|  additives_list|      Product_name|   Additives|
+---------+----------------+------------------+------------+
|     E322|E322 - Lecithins|Belgische Pralinen|   Lecithins|
|     E420| E420 - Sorbitol|Belgische Pralinen|(i) Sorbitol|
|     E422| E422 - Glycerol|Belgische Pralinen|    Glycerol|
+---------+----------------+------------------+------------+
only showing top 3 rows



In [72]:
#Potential danger products ?
non_registeredA = health_df.filter(col("Additives").isNull()).select("Product_name","additives_list")
non_registeredA.show(10)
print("As a ratio against all products")
c = non_registeredA.count()
print(100 * c / productsid)
print("total products with unkown additives ")
non_registeredA.count()

+--------------------+--------------------+
|        Product_name|      additives_list|
+--------------------+--------------------+
|  Sour Fruit Gummies|     E428 - Gelatine|
|       Vita Cola pur|E150d - Sulphite ...|
|Chocolate and car...|     E1400 - Dextrin|
|       Zesty Italian|     E160b - Annatto|
|       Bröd Mjukkaka|E472e - Mono- and...|
|Sweet Baby Ray's ...|E150d - Sulphite ...|
|Betty Crocker Sca...|    E160a - Carotene|
|Betty Crocker Sca...|E160ai - Beta-car...|
|       4 Vol Au Vent|    E160a - Carotene|
|Wild berry candie...|     E1400 - Dextrin|
+--------------------+--------------------+
only showing top 10 rows

As a ratio
6.859509565446059


5217